In [12]:
from qdrant_client import QdrantClient, models
import minsearch
from groq import Groq
import json
import os
from qdrant_client.http.models import VectorParams, Distance, PointStruct, SearchRequest


In [14]:
client = Groq(api_key=os.getenv("GROQ_API_KEY"))


QUESTIION 1

In [8]:
from fastembed.embedding import TextEmbedding
import numpy as np

# Load the embedding model
embedder = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en", cache_dir=".cache", threads=2)

# Input query
query = ["I just discovered the course. Can I join now?"]

# Generate embedding (returns a generator, so convert to list)
embedding = list(embedder.embed(query))[0]  # get first (and only) embedding

# Convert to numpy array
embedding_np = np.array(embedding)

# Print minimal value
min_val = np.min(embedding_np)
print("Minimal value:", min_val)


Minimal value: -0.11726373885183883


In [9]:
import numpy as np
np.linalg.norm(query)

QUESTION 2

In [11]:
from fastembed.embedding import TextEmbedding
import numpy as np

# Load model
embedder = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

# Define inputs
query = ["I just discovered the course. Can I join now?"]
doc = ["Can I still join the course after the start date?"]

# Embed both
query_vec = np.array(list(embedder.embed(query))[0])
doc_vec = np.array(list(embedder.embed(doc))[0])

# Compute cosine similarity
cos_sim = np.dot(query_vec, doc_vec) / (np.linalg.norm(query_vec) * np.linalg.norm(doc_vec))
print("Cosine similarity:", cos_sim)


Cosine similarity: 0.9008528895674548


QUESTION 3

In [2]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [3]:
documents[0]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [4]:
from fastembed.embedding import TextEmbedding
import numpy as np

# Load the model
embedder = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

# Extract document texts
doc_texts = [doc["text"] for doc in documents]

# Get document embeddings and stack into matrix V (shape: 5 x 512)
V = np.vstack([np.array(emb) for emb in embedder.embed(doc_texts)])

# Get the query embedding
query_vec = np.array(list(embedder.embed(["I just discovered the course. Can I join now?"]))[0])

# Normalize embeddings
V_norm = V / np.linalg.norm(V, axis=1, keepdims=True)
query_norm = query_vec / np.linalg.norm(query_vec)

# Compute cosine similarities using dot product
cos_similarities = V_norm @ query_norm

# Get the most similar index
most_similar_index = int(np.argmax(cos_similarities))
print("Most similar document index:", most_similar_index)


2025-06-26 17:46:08.339 | WARNING  | fastembed.embedding:<module>:7 - DefaultEmbedding, FlagEmbedding, JinaEmbedding are deprecated.Use from fastembed import TextEmbedding instead.


Most similar document index: 1


QUESTION 4

In [6]:
# Step 1: Create full_text field by concatenating question + text
full_texts = [doc['question'] + ' ' + doc['text'] for doc in documents]

# Step 2: Embed all full_text fields
full_embeddings = np.vstack([np.array(emb) for emb in embedder.embed(full_texts)])

# Step 3: Normalize
full_embeddings_norm = full_embeddings / np.linalg.norm(full_embeddings, axis=1, keepdims=True)
query_norm = query_vec / np.linalg.norm(query_vec)

# Step 4: Cosine similarities
cosine_similarities = full_embeddings_norm @ query_norm

# Step 5: Find most similar index
most_similar_index = int(np.argmax(cosine_similarities))
print("Most similar document index (Q4):", most_similar_index)


Most similar document index (Q4): 0


QUESTION 5

In [7]:

# Step 1: Initialize the smaller embedding model
embedder = TextEmbedding(model_name="BAAI/bge-small-en")

# Step 2: Create full_text field by concatenating question + text
full_texts = [doc['question'] + ' ' + doc['text'] for doc in documents]

# Step 3: Embed the documents
doc_embeddings = np.vstack([np.array(emb) for emb in embedder.embed(full_texts)])

# Step 4: Embed the query
query = "I just discovered the course. Can I join now?"
query_vec = np.array(list(embedder.embed([query]))[0])

# Step 5: Normalize embeddings
doc_embeddings_norm = doc_embeddings / np.linalg.norm(doc_embeddings, axis=1, keepdims=True)
query_vec_norm = query_vec / np.linalg.norm(query_vec)

# Step 6: Compute cosine similarities using matrix dot product
cosine_similarities = doc_embeddings_norm @ query_vec_norm

# Step 7: Find the most similar document index
most_similar_index = int(np.argmax(cosine_similarities))

# Step 8: Print results
print("Embedding dimensionality:", doc_embeddings.shape[1])  # Should print 384


Fetching 5 files:   0%|                                                                                          | 0/5 [00:00<?, ?it/s]C:\Users\HP\Documents\LLM_Zoomcamp project\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\AppData\Local\Temp\fastembed_cache\models--Qdrant--bge-small-en. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-devic

Embedding dimensionality: 384
Most similar document index: 0


QUESTION 6

In [9]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [13]:
embedder = TextEmbedding(model_name="BAAI/bge-small-en")

qdrant = QdrantClient(":memory:")  
collection_name = "ml_zoomcamp_faq"
dim = 384 
qdrant.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=dim, distance=Distance.COSINE),
)

texts = [doc['question'] + ' ' + doc['text'] for doc in documents]
vectors = [np.array(vec).tolist() for vec in embedder.embed(texts)]

payloads = [{'text': text} for text in texts]

points = [
    PointStruct(id=i, vector=vectors[i], payload=payloads[i])
    for i in range(len(vectors))
]

qdrant.upsert(collection_name=collection_name, points=points)

query = "I just discovered the course. Can I join now?"
query_vec = np.array(list(embedder.embed([query]))[0]).tolist()

## Search in Qdrant
search_result = qdrant.search(
    collection_name=collection_name,
    query_vector=query_vec,
    limit=1,
)

top_score = search_result[0].score
print("Highest score from Qdrant search:", top_score)

C:\Users\HP\AppData\Local\Temp\ipykernel_3688\1642240905.py:10: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


Highest score from Qdrant search: 0.8703173344744922


C:\Users\HP\AppData\Local\Temp\ipykernel_3688\1642240905.py:33: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = qdrant.search(


In [2]:

client = Groq(api_key=os.getenv("GROQ_API_KEY"))
